# Dihydrofolate Reductase (DHFR) Inhibitors Classification


## Binary Activity Classifier

This Jupyter notebook takes quantitative structure-activity relationship (QSAR) data for a compiled set of dihydrofolate reductase (DHFR) inhibitors to construct a binary classifier to predict active/inactive compounds.

## Dataset

https://www.kaggle.com/shashwatwork/dihydrofolate-reductase-inhibitors-data-dhfr

## Motivation

Early phase drug discovery involves the identification of compounds with therapeutic potential for development (hit discovery). Such candidates must engage the biological target with reasonable selectivity and possess favourable physicochemical properties such as solubility, cell permeability, and metabolic stability. A traditional early discovery phase protocol involves high-throughput screening (HTC) of compound libraries and determining their corresponding affinities via biological assays. While this can be effective, a notable limitation is the arduous task of conducting experiments for an enormous number of compounds which is expensive and slow. With the advent of computational power and sufficiently accurate physics-based models, computer (in silico) simulation and calculations are often used to supplement experimental assays. These calculations may incorporate multiple facets of compound binding, including the compound polarity, atomic bond lengths, atomic angles, etc. and collectively constitute QSAR data. Modern hit discovery utilizes this quantitative data to drive hypotheses and steer experimental efforts towards compounds which are most promising, thus enhancing drug discovery. 

Suppose you are given a dataset of historical DHFR inhibitors, denoted active or inactive, with accompanying QSAR data. You are working within a large hit discovery team tasked with designing novel DHFR inhibitors with better binding affinities and physicochemical properties. Given the historical DHFR dataset, you want to construct a classifier to determine whether new compounds are active or inactive. This would eliminate the need for biological assay determination for all compounds (assuming the classifier is sufficiently accurate), saving time, and increasing the ability to probe chemical space (i.e. evaluate diverse compounds which can be advantageous when trying to optimize physicochemical properties). 


## Objective

Use the DHFR inhibitors dataset to construct a binary classifier to predict active/inactive inhibitor compounds.

Import standard data processing and visualization libraries.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

DATA_PATH = "/kaggle/input/dihydrofolate-reductase-inhibitors-data-dhfr/dhfr.csv"
dhfr_data = pd.read_csv(DATA_PATH)

In [ ]:
dhfr_data.head()

In [ ]:
dhfr_data.info()

228/229 column data series consists of "float64" data type. Referring to above head method, we can identify the "object" to belong to column "Y". Moreover, since the data was loaded using pandas' read_csv method, "object" must be text. We confirm this above where "Y" values are either "active" or "inactive".

In [ ]:
dhfr_data.describe()

Plot histograms to visualize the data distribution.

In [ ]:
#qsar_metric_names = dhfr_data.columns.values

#sb.set_theme()

#for name in qsar_metric_names:
    #dhfr_data[name].hist()
    #plt.title(name)
    #plt.xlabel("QSAR Feature Value")
    #plt.ylabel("Total Count")
    #plt.figure()

Plot the histogram of the distribution of the "Y" label, inhibitor activity again. We can obtain a starting metric representing the best performance of a classifier that predicts activity at random. In the cell block below, we see that 203/325 (majority label) entries are "active" inhibitors. Therefore, a classifer that predicts "active" for every inhibitor could achieve ~62.5% accuracy (203/325). The classifier must achieve a greater accuracy.

In [ ]:
dhfr_data["Y"].hist()
plt.title("DHFR Inhibitors")
plt.xlabel("Inhibitor Activity")
plt.ylabel("Total Amount")
plt.figure()

print(dhfr_data["Y"].value_counts())
# best classifier performance if it were to guess "active" every time
print(f'\nBest classifier performance if it predicts "active" for all inhibitors: {203/325}')

## Dataset Splitting

To prepare for model training, let's split the dataset into 3 sets: train/validation/test (60/20/20). The validation set will be used during hyperparameter fine-tuning. It is important that the test set is left untouched until model evaluation to obtain a true estimate of the model's ability to generalize.

In [ ]:
from sklearn.model_selection import train_test_split

# extract the features and the label
X = np.array(dhfr_data.drop("Y", axis=1))
y = np.array(dhfr_data["Y"])

# generate the train/test sets following an 80/20 split
# random_state=42 for reproducibility and shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# split the train sets further to generate a validations set
# following an overall train/validation/test splitting of 60/20/20
# random_state=42 for reproducibility
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# verify the splitting is 60/20/20 
print(f"Training set proportion: {len(X_train)/325}")
print(f"Validation set proportion: {len(X_val)/325}")
print(f"Test set proportion: {len(X_test)/325}")

## Feature Processing

Support Vector Machine (SVM) will be one of the models we fit. It is sensitive to feature scales and therefore, it is a good idea to standardize the quantitative features. In addition, we must handle the categorical label, "Y" which again denotes either "active" or "inactive". 

It is important that all the feature matrices (X_train, X_val, X_test) are standardardized. However, attention needs to be given to standardizing all the sets with the mean and standard deviation of X_train only. This is to avoid data leakage that could bias the X_val and X_test into giving better generalization errors than reality.

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scalar = StandardScaler()

# standardize X_train with its mean and standard deviation 
X_train = std_scalar.fit_transform(X_train)
# use only transform so the X_train mean and standard deviation is maintained
X_val = std_scalar.transform(X_val)
X_test = std_scalar.transform(X_test)

# write a function to binarize the label matrices instead of using a sklearn class such as LabelBinarizer
# this is to control the value mappings to follow the convention in drug discovery
# "active" -> 1 and "inactive" -> 0
def activity_enc(label_matrix: np.array) -> np.array:
    
    binarized_label = []
    
    for label in label_matrix:
        if label == "inactive":
            binarized_label.append(0)
        if label == "active":
            binarized_label.append(1)
            
    return np.array(binarized_label)

y_train = activity_enc(y_train)
y_val = activity_enc(y_val)
y_test = activity_enc(y_test)

# verify the labels are binarized as intended
print(y_train)

## Model Training: LinearSVC, SVC Polynomial Kernel, Random Forest

We are now ready for model training! Fit LinearSVC, Polynomial SVC, and Random Forest models for binary classification.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# LinearSVC: use default hyperparameters except max_iter to ensure convergence
lin_svc = LinearSVC(C=1, loss="hinge", max_iter=10000)
# Polynomial SVC
poly_svc = SVC(kernel="poly", degree=3, coef0=1, C=1, probability=True)
# Random Forest: random_state=42 for reproducibility
rand_forest = RandomForestClassifier(random_state=42)

# fit models
lin_svc.fit(X_train, y_train.ravel())
poly_svc.fit(X_train, y_train.ravel())
rand_forest.fit(X_train, y_train.ravel())

# obtain the train, val, and test set accuracies for all models
lin_svc_train_score = lin_svc.score(X_train, y_train)
lin_svc_val_score = lin_svc.score(X_val, y_val)
lin_svc_test_score = lin_svc.score(X_test, y_test)

poly_svc_train_score = poly_svc.score(X_train, y_train)
poly_svc_val_score = poly_svc.score(X_val, y_val)
poly_svc_test_score = poly_svc.score(X_test, y_test)

rand_forest_train_score = rand_forest.score(X_train, y_train)
rand_forest_val_score = rand_forest.score(X_val, y_val)
rand_forest_test_score = rand_forest.score(X_test, y_test)

# contruct a DataFrame to display model accuracy performance
performance_df = pd.DataFrame({
"Model": ["LinearSVC", "SVC Poly Kernel", "Random Forest"],
"Train Acc.": [lin_svc_train_score, poly_svc_train_score, rand_forest_train_score],
"Val. Acc.": [lin_svc_val_score, poly_svc_val_score, rand_forest_val_score],
"Test Acc.": [lin_svc_test_score, poly_svc_test_score, rand_forest_test_score],
"Val. Error": 
[f"{round((((lin_svc_train_score-lin_svc_val_score))/lin_svc_train_score*100),2)}%",
f"{round((((poly_svc_train_score-poly_svc_val_score))/poly_svc_train_score*100),2)}%",
f"{round((((rand_forest_train_score-rand_forest_val_score))/rand_forest_train_score*100),2)}%"],
"Test Error":
[f"{round((((lin_svc_train_score-lin_svc_test_score))/lin_svc_train_score*100),2)}%",
f"{round((((poly_svc_train_score-poly_svc_test_score))/poly_svc_train_score*100),2)}%",
f"{round((((rand_forest_train_score-rand_forest_test_score))/rand_forest_train_score*100),2)}%"]})

print(performance_df.to_string(index=False))

In general, the accuracy on the validation and test sets is high for all 3 models. However, there is notable generalization error in the case of the LinearSVC and Random Forest models, suggesting overfitting. It is perhaps not surprising that the Random Forest model achieved perfect accuracy on the training set with decreased accuracies on the validation and test sets as no hyperparameter limitations were enforced during fitting. Consequently, the model was given total freedom in tree depth, nodes, etc. which is prone to lead to overfitting. Let's use cross-validation to investigate what the average generalization error is. This would give a better estimate of the generalization error to give us a starting point to improve the models. The goal is to train a model whose performance on the test set is as close as possible to the training set performance, thus minimizing generalization error.  

First, re-generate the train/test sets following at 80/20 split. The validation set is not explictly generated because subsequent cross-validation takes care of that.

In [ ]:
# random_state=42 for reproducibility and shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# perform the same feature processing as before
X_train = std_scalar.fit_transform(X_train)
X_test = std_scalar.transform(X_test)

y_train = activity_enc(y_train)
y_test = activity_enc(y_test)

# verify the splitting is 80/20
print(f"Training set proportion: {len(X_train)/325}")
print(f"Test set proportion: {len(X_test)/325}")

In [ ]:
from sklearn.model_selection import cross_val_score

# use cv=4 so the validation set represents 20% of the overall dataset 
# since the train set is 80% of the overall dataset
lin_svc_cross_val_scores = cross_val_score(lin_svc, X_train, y_train.ravel(), cv=4)
poly_svc_cross_val_scores = cross_val_score(poly_svc, X_train, y_train.ravel(), cv=4)
rand_forest_cross_val_scores = cross_val_score(rand_forest, X_train, y_train.ravel(), cv=4)

print(f"LinearSVC Cross-Validation Validation Set Scores: {lin_svc_cross_val_scores}")
print(f"LinearSVC Cross-Validation Validation Set Scores Mean: {lin_svc_cross_val_scores.mean()}\n")
print(f"Poly SVC Cross-Validation Validation Set Scores: {poly_svc_cross_val_scores}")
print(f"Poly SVC Cross-Validation Validation Set Scores Mean: {poly_svc_cross_val_scores.mean()}\n")
print(f"Random Forest Cross-Validation Validation Set Scores: {rand_forest_cross_val_scores}")
print(f"Random Forest Cross-Validation Validation Set Scores Mean: {rand_forest_cross_val_scores.mean()}")

The cross-validation scores differ based on the different training folds used, as expected. The mean of all the LinearSVC models is ~ 88.46% which is quite similar to the generalization error when we trained the model above with the pre-generated and constant validation set (as opposed to cross-validation folds validation sets). In contrast, the mean cross-validation score for the Poly SVC and Random Forest models is ~ 93.08 for both which is notably higher. Morevoer, in 1 case for the Ramdom Forest model, the validation accuracy is >95%. Taken together, these observations suggest that the LinearSVC model is overfitting, the Poly SVC performs better and only slightly overfits, and the Random Forest model although overfitting, can perform well given better training subsets.

The performance of all 3 models may be improved following hyperparameter optimization which can also reduce overfitting:

1) LinearSVC
* The hyperparameter, C controls the width of the support vector margins
* In general, reducing C widens the support vector margins and can decrease overfitting
* Adjusting C in our case will probably not reduce the generalization error as the model has already been trained
  with C=1 (default and lowest possible value of C)
  
2) Poly SVC
* The hyperparameter, C controls the width of the support vector margins
* The hyperparameter, degree controls the degree of polynomialization which adds features.
  High polynomial degrees can increase accuracy but leads to combinatorial explosion and can cause overfitting.
  Reducing the polynomial degree can reduce overfitting
* coef0 controls the polynomial features' affect on the model performance
* Since the Poly SVC model generalizes relatively well, the accuracy could be improved by increasing the polynomial degree   with a relatively lower risk of overfitting 

3) Random Forest
* Many hyperparameters --> below are 2 important ones
* n_estimators denoting the number of trees 
* max_features denoting the max number of features to consider when splitting 
* The majority of the hyperparameters used to train the Random Forest were static (ex. n_estimators was kept default     which is 100 from sklearn V0.22+)
* Since the Random Forest model is slightly overfitting, tuning the hyperparameters could reduce the generalization     error and lead to an overall better classifier

Let's try tuning the 3 models.

In [ ]:
from sklearn.model_selection import GridSearchCV

# LinearSVC hyperparameters to try
lin_svc_param_grid = [{"C": list(range(1,11))}]

# Poly SVC hyperparameter combinations to try
poly_svc_param_grid = [{"C": list(range(1,11)), 
                        "degree": [2, 3, 4, 5, 6, 7, 8, 9, 10],
                        "coef0": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,
                                  1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]}]

# Random Forest hyperparameter combinations to try (there are many other hyperparameters that are not tried here)
rand_forest_param_grid = [{"n_estimators": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                           "max_features": [2, 3, 4, 5, 6, 7, 8, 9, 10]}]

# Fit all 3 models to find the best hyperparameters (cv=4 so the validation set is 20% of the overall dataset)
lin_svc_gscv = GridSearchCV(lin_svc, param_grid=lin_svc_param_grid, cv=4)
poly_svc_gscv = GridSearchCV(poly_svc, param_grid=poly_svc_param_grid, cv=4)
rand_forest_gscv = GridSearchCV(rand_forest, param_grid=rand_forest_param_grid, cv=4)

# re-train all 3 models with the best hyperparameters found
lin_svc_gscv.fit(X_train, y_train.ravel())
poly_svc_gscv.fit(X_train, y_train.ravel())
rand_forest_gscv.fit(X_train, y_train.ravel())

# display the best hyperparameters from the combinations tried
print(f"LinearSVC Best Hyperparameters Found: {lin_svc_gscv.best_params_}")
print(f"Poly SVC Best Hyperparameters Found: {poly_svc_gscv.best_params_}")
print(f"Random Forest Best Hyperparameters Found: {rand_forest_gscv.best_params_}\n")

The best hyperparameters found are displayed. Importantly, none of the hyperparameter values are at the extreme values tried (ex. Random Forest max_features=6 out of the possible vaues 2-10). If the fine-tuned hyperparameters were at the extreme values (ex. Random Forest max_features=10), then it could be worthwhile to expand the hyperparameter search space.

Let's next evaluate the fine-tuned models on the test set and generate a DataFrame to compare the models' performances pre-fine-tuning and post-fine-tuning.

In [ ]:
# extract the performance score of all 3 models on the test set
lin_svc_gscv_test_score = lin_svc_gscv.score(X_test, y_test)
poly_svc_gscv_test_score = poly_svc_gscv.score(X_test, y_test)
rand_forest_gscv_test_score = rand_forest_gscv.score(X_test, y_test)

# contruct a DataFrame to compare all 3 models' performance following hyperparameter fine-tuning
fine_tuned_performance_df = pd.DataFrame({
"Model": ["LinearSVC", "SVC Poly Kernel", "Random Forest"],
"Prev. Train Acc.": [lin_svc_train_score, poly_svc_train_score, rand_forest_train_score],
"Prev. Test Acc.": [lin_svc_test_score, poly_svc_test_score, rand_forest_test_score],
"Prev. Test Error":
[f"{round((((lin_svc_train_score-lin_svc_test_score))/lin_svc_train_score*100),2)}%",
f"{round((((poly_svc_train_score-poly_svc_test_score))/poly_svc_train_score*100),2)}%",
f"{round((((rand_forest_train_score-rand_forest_test_score))/rand_forest_train_score*100),2)}%"],
"New Train Acc.": [lin_svc_gscv.best_score_, poly_svc_gscv.best_score_, rand_forest_gscv.best_score_],
"New Test Acc.": [lin_svc_gscv_test_score, poly_svc_gscv_test_score, rand_forest_gscv_test_score],
"New Test Error":
[f"{round((((lin_svc_gscv.best_score_-lin_svc_gscv_test_score))/lin_svc_gscv.best_score_*100),2)}%",
f"{round((((poly_svc_gscv.best_score_-poly_svc_gscv_test_score))/poly_svc_gscv.best_score_*100),2)}%",
f"{round((((rand_forest_gscv.best_score_-rand_forest_gscv_test_score))/rand_forest_gscv.best_score_*100),2)}%"]})

print(fine_tuned_performance_df.to_string(index=False))                   

The generalization errors for all 3 models improved a lot! 

1) LinearSVC
* The LinearSVC performance on the test set stayed the same 
* The fine-tuned LinearSVC training accuracy decreased but this is due to taking the average training score from 
  c=4 cross-validation as opposed to training using the entire training set. The more important comparison is the
  relative performance on the test set and we see that the model now generalizes much better. In fact, the "New Test     Error" is negative because the performance on the test set exceeds the training set.
  
2) Poly SVC
* Similar to LinearSVC, the training accuracy decreased. However, the test accuracy is identical to the previous test   accuracy, reducing the generalization error.
* The slight overfitting previously has been reduced (at least for this test set. This is no guarantee the model will perform as well on completely new data.)

3) Random Forest
* Training accuracy decreased but test accuracy increased, yielding a much lower generalization error (2.04%)
* The model was overfitting before (again, maybe expected as the hyperparameters were given total freedom for tree       growth) but now we have reduced the overfitting to yield a model that can generalize much better

Based on these metrics alone, it seems that the Random Forest would be the model of choice given the best test set accuracy and generalization. However, it is useful to look at metrics beyond accuracy especially as this is a classification task. Let's take a look at the confusion matrix along with the precision and recall scores. These metrics will offer further insights into the models' performances.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

# obtain the predictions on X_train using the fine-tuned hyperparameters
lin_svc_gscv_y_train_pred = lin_svc_gscv.predict(X_train)
poly_svc_gscv_y_train_pred = poly_svc_gscv.predict(X_train)
rand_forest_gscv_y_train_pred = rand_forest_gscv.predict(X_train)

print(f"Fine-Tuned LinearSVC X_train Confusion Matrix: \n {confusion_matrix(y_train, lin_svc_gscv_y_train_pred)}")
print(f"Average Cross-Validation Accuracy: {lin_svc_gscv.best_score_}")
print(f"Precision Score: {precision_score(y_train, lin_svc_gscv_y_train_pred)}")
print(f"Recall Score: {recall_score(y_train, lin_svc_gscv_y_train_pred)}\n")

print(f"Fine-Tuned Poly SVC X_train Confusion Matrix: \n {confusion_matrix(y_train, poly_svc_gscv_y_train_pred)}")
print(f"Average Cross-Validation Accuracy: {poly_svc_gscv.best_score_}")
print(f"Precision Score: {precision_score(y_train, poly_svc_gscv_y_train_pred)}")
print(f"Recall Score: {recall_score(y_train, poly_svc_gscv_y_train_pred)}\n")

print(f"Fine-Tuned Random Forest X_train Confusion Matrix: \n {confusion_matrix(y_train, rand_forest_gscv_y_train_pred)}")
print(f"Average Cross-Validation Accuracy: {rand_forest_gscv.best_score_}")
print(f"Precision Score: {precision_score(y_train, rand_forest_gscv_y_train_pred)}")
print(f"Recall Score: {recall_score(y_train, rand_forest_gscv_y_train_pred)}\n")

The precision and recall scores for all 3 models trained with the fine-tuned hyperparameters are nearly perfect for the training set! Note that there seems to be a discrepancy between the accuracy and the precision and recall scores. For example, the LinearSVC confusion matrix only displays 1 misclassification which is reflected correctly in the Recall score < 1. However, how is it that the "accuracy" is ~88.5% if the model only made 1 misclassification? The reason is because the accuracy score represents the average accuracy score obtained following cross-validation. Recall the training set which made up 80% of the overall dataset was fed into GridSearchCV. Cross-validation used 4 folds and therefore, each model was trained on some combination of 3 folds (60% of the training data) and validated on the remaining fold (20% of the training data). The accuracy represents the average accuracy of this cross-validation. Moreover, once the optimal hyperparameters were determined by GridSearchCV, the models were re-fit using the entire training set (80% of the overall data). These two factors taken together explain the "discrepancy" in the accuracy. In reality, GridSearchCV improved the models as shown in the previous DataFrame comparing the generalization errors pre and post hyperparameter fine-tuning. Overfitting was reduced and the models generalize on new data better.

Let's now calculate the precision and recall scores on the test data and observe how the training set performance carries over to test set performance.

In [ ]:
# obtain the predictions on X_test using the fine-tuned hyperparameters
lin_svc_gscv_y_test_pred = lin_svc_gscv.predict(X_test)
poly_svc_gscv_y_test_pred = poly_svc_gscv.predict(X_test)
rand_forest_gscv_y_test_pred = rand_forest_gscv.predict(X_test)

print(f"Fine-Tuned LinearSVC X-test Confusion Matrix: \n {confusion_matrix(y_test, lin_svc_gscv_y_test_pred)}")
print(f"Average Cross-Validation Accuracy: {lin_svc_gscv.best_score_}")
print(f"Test Set Accuracy: {lin_svc_gscv_test_score}")
print(f"Precision Score: {precision_score(y_test, lin_svc_gscv_y_test_pred)}")
print(f"Recall Score: {recall_score(y_test, lin_svc_gscv_y_test_pred)}\n")

print(f"Fine-Tuned Poly SVC X_test Confusion Matrix: \n {confusion_matrix(y_test, poly_svc_gscv_y_test_pred)}")
print(f"Average Cross-Validation Accuracy: {poly_svc_gscv.best_score_}")
print(f"Test Set Accuracy: {poly_svc_gscv_test_score}")
print(f"Precision Score: {precision_score(y_test, poly_svc_gscv_y_test_pred)}")
print(f"Recall Score: {recall_score(y_test, poly_svc_gscv_y_test_pred)}\n")

print(f"Fine-Tuned Random Forest X_test Confusion Matrix: \n {confusion_matrix(y_test, rand_forest_gscv_y_test_pred)}")
print(f"Average Cross-Validation Accuracy: {rand_forest_gscv.best_score_}")
print(f"Test Set Accuracy: {rand_forest_gscv_test_score}")
print(f"Precision Score: {precision_score(y_test, rand_forest_gscv_y_test_pred)}")
print(f"Recall Score: {recall_score(y_test, rand_forest_gscv_y_test_pred)}\n")

The precision and recall scores for all 3 models have decreased which is to be expected. In general, they are still very good. Note that the accuracy score is the true accuracy displayed by the confusion matrix and not the average accuracy from cross-validation as was the case in the cell block above. This is because the test set was untouched during training and was not fed into GridSearchCV.

Taking the accuracy, precision, and recall metrics together provides insight into choosing the most suitable model to deploy. 

1) LinearSVC
* Lowest cross-validation accuracy 
* Lowest/2nd lowest test set accuracy 
* Lowest test set precision
* Highest/2nd highest test set recall

2) Poly SVC
* 2nd highest cross-validation accuracy
* Lowest/2nd lowest test set accuracy 
* 2nd highest test set precision
* Lowest recall

3) Random Forest
* Highest cross-validation accuracy
* Highest test set accuracy 
* Highest test set precision
* Highest recall

Based on the observations on accuracy and generalization error, one would probably choose the Random Forest model over the LinearSVC and Poly SVC. This choice is further supported by the test set precision and recall scores where the Random Forest outperforms the other two models.

As a final note, if the precision and recall of the models were much lower, one could look at tuning these models by increasing either the precision or recall (based on the objective of the classifier), which exhibit an inverse relationship. From a drug discovery perspective, it is probably preferable to maximize recall (minimize the number of "active" inhibitors missed and let a few false positives through). Doing so will curate a library of presumably "active" compounds which can then be further filtered (ex. based on synthetic feasibility and/or physicochemical properties). This of course assumes a reasonable precision is maintained or else we run into the problem of wasting time and resources to test inhibitors which have a high probability of being "inactive". 

To visualize the precision-recall trade-off, let's plot precision-recall curves.

In [ ]:
from sklearn.metrics import precision_recall_curve

rand_forest_gscv_y_decision_scores = rand_forest_gscv.predict_proba(X_train)
poly_svc_gscv_y_decision_scores = poly_svc_gscv.predict_proba(X_train)

poly_svc_gscv_precision, poly_svc_gscv_recall, poly_svc_gscv_thresholds = precision_recall_curve(y_train, poly_svc_gscv_y_decision_scores[:, 1])
rand_forest_gscv_precision, rand_forest_gscv_recall, rand_forest_gscv_thresholds = precision_recall_curve(y_train, rand_forest_gscv_y_decision_scores[:, 1])

def precision_recall_curve(precision_1, recall_1, precision_2, recall_2, clf_name, clf2_name):
    plt.figure(figsize=(12, 8)); plt.rc("xtick", labelsize=16); plt.rc("ytick", labelsize=16)
    plt.plot(recall_1, precision_1, "y-", linewidth=2, label=clf_name)
    plt.plot(recall_2, precision_2, "g-", linewidth=2, label=clf2_name)
    plt.title("Precision-Recall Curve", fontsize=18)
    plt.xlabel("Recall", fontsize=18)
    plt.ylabel("Precision", fontsize=18)
    plt.legend(fontsize=18)
    plt.plot([0.96, 0.96], [0.96, 1], "r--o")
    plt.figure()

precision_recall_curve(poly_svc_gscv_precision, poly_svc_gscv_recall, 
                       rand_forest_gscv_precision, rand_forest_gscv_recall,
                       "Poly SVC", "Random Forest")

The precision-recall curves are not as interesting since the models already perform so well. However, it nicely illustrates the precision-recall trade-off. In the case of the Poly SVC model, the training precision and recall scores are as follows:

Precision Score: 0.9651162790697675
Recall Score: 1.0

If we wanted to increase precision to 1.0, the recall would drop to ~0.96 as shown by the red-dotted line in the plot. Note this is not desired for the inhibitor activity classifier as we want to maximize recall. 

Let's next generate Receiver Operating Characteristic (ROC) curves.

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

poly_svc_gscv_y_decision_scores = poly_svc_gscv.predict_proba(X_train)
rand_forest_gscv_y_decision_scores = rand_forest_gscv.predict_proba(X_train)

poly_svc_gscv_false_pos_rates, poly_svc_gscv_true_pos_rates, poly_svc_gscv_thresholds = roc_curve(y_train, poly_svc_gscv_y_decision_scores[:, 1])
rand_forest_gscv_false_pos_rates, rand_forest_gscv_true_pos_rates, rand_forest_gscv_thresholds = roc_curve(y_train, rand_forest_gscv_y_decision_scores[:, 1])

def roc_curve(false_pos_rates_1, true_pos_rates_1, false_pos_rates_2, true_pos_rates_2, clf_name, clf2_name):
    plt.figure(figsize=(12, 8)); plt.rc("xtick", labelsize=16); plt.rc("ytick", labelsize=16)
    plt.plot(false_pos_rates_1, true_pos_rates_1, "y-", linewidth=5, label=clf_name)
    plt.plot(false_pos_rates_2, true_pos_rates_2, "g-", linewidth=5, label=clf2_name)
    plt.title("Receiver Operating Characteristic (ROC) Curve", fontsize=18)
    plt.xlabel("False Positive Rate", fontsize=18)
    plt.ylabel("True Positive Rate (Recall)", fontsize=18)
    plt.legend(fontsize=18, loc="lower right")
    plt.axis([0, 1, 0, 1]) 
    plt.figure()
    
roc_curve(poly_svc_gscv_false_pos_rates, poly_svc_gscv_true_pos_rates, 
          rand_forest_gscv_false_pos_rates, rand_forest_gscv_true_pos_rates, 
          "Poly SVC", "Random Forest")
print(f"Poly SVC ROC AUC Score: {roc_auc_score(y_train, poly_svc_gscv_y_decision_scores[:, 1])}")
print(f"Random Forest ROC AUC Score: {roc_auc_score(y_train, rand_forest_gscv_y_decision_scores[:, 1])}")

Once again the ROC curves are not as interesting since the models perform so well. The perfect classifier would be to the top left of the plot which is achieved by the Random Forest model. "Perfect" here is defined by the performance on the train set. As we observed previously, the Random Forest model does not generalize as well and its performance is certainly not perfect on the test set. Nonetheless, both models perform well on the train set and is quantitatively demonstrated by their Area Under Curve (AUC) scores. Based on the scores alone, the Random Forest model is just so ever slightly better than the Poly SVC model on the train set.

Finally, we re-visit the Random Forest as this is the model we choose to deploy. It possesses the best cross-validation accuracy, the best generalization performance, and the best recall and precision on the test set. These metrics are displayed one final time in the cell block below.

In [ ]:
# contruct a DataFrame to compare all 3 models' performance following hyperparameter fine-tuning
fine_tuned_performance_df = pd.DataFrame({
"Model": ["Random Forest"],
"Fine-Tuned Train Acc.": [rand_forest_gscv.best_score_],
"Fine-Tuned Test Acc.": [rand_forest_gscv_test_score],
"Fine-Tuned Test Error": 
f"{round((((rand_forest_gscv.best_score_-rand_forest_gscv_test_score))/rand_forest_gscv.best_score_*100),2)}%"})
                                
print(fine_tuned_performance_df.to_string(index=False))

print(f"\nFine-Tuned Random Forest X_train Confusion Matrix: \n {confusion_matrix(y_train, rand_forest_gscv_y_train_pred)}")
print(f"Train Set Precision Score: {precision_score(y_train, rand_forest_gscv_y_train_pred)}")
print(f"Train set Recall Score: {recall_score(y_train, rand_forest_gscv_y_train_pred)}\n")

print(f"Fine-Tuned Random Forest X_test Confusion Matrix: \n {confusion_matrix(y_test, rand_forest_gscv_y_test_pred)}")
print(f"Test Precision Score: {precision_score(y_test, rand_forest_gscv_y_test_pred)}")
print(f"Test Recall Score: {recall_score(y_test, rand_forest_gscv_y_test_pred)}\n")
